In [1]:
using SaddleSearch 
using SaddleSearch.TestSets
using Plots 
using Dierckx

In [56]:
using Dierckx
export StringMethod2

"""
`StringMethod`: the most basic string method variant, minimising the energy
normally to the string by successive steepest descent minimisations at fixed
step-size with an intermediate redistribution of the nodes.

### Parameters:
* `alpha` : step length
* `tol_res` : residual tolerance
* `maxnit` : maximum number of iterations
* `precon` : preconditioner
* `precon_prep!` : update function for preconditioner
* `verbose` : how much information to print (0: none, 1:end of iteration, 2:each iteration)
* `precon_cond` : true/false whether to precondition the minimisation step
"""
type StringMethod2
   alpha::Float64
   # ------ shared parameters ------
   tol_res::Float64
    tol_resF::Float64
   maxnit::Int
   precon
   precon_prep!
   verbose::Int
   precon_cond::Bool
end


function run2!{T}(method::StringMethod2, E, dE, x0::Vector{T}, t0::Vector{T})
      # read all the parameters
    alpha = method.alpha
    tol_res = method.tol_res
#     tol_resF = method.tol_resF
    maxnit = method.maxnit
    precon = method.precon
    precon_prep! = (P, x) -> P
    verbose = 2
    precon_cond = false
    
    P=precon
      # initialise variables
      x, t = copy(x0), copy(t0)
      nit = 0
      numdE, numE = 0, 0
      log = SaddleSearch.IterationLog()
      # and just start looping
      if verbose >= 2
        @printf(" nit |  sup|∇E|_∞   ||∇Eᵏ|_∞-|∇Eᵏ⁺¹|_∞|\n")
        @printf("-----|---------------------------------\n")
      end
      for nit = 0:maxnit
         # normalise t
         P = precon_prep!(P, x)
         t ./= [sqrt(dot(t[i], P, t[i])) for i=1:length(x)]
         # evaluate gradients, and more stuff
         dE0 = [dE(x[i]) for i=1:length(x)]
         t[1] =zeros(length(t[1])); t[length(x)]=zeros(length(t[1]));
         dE0perp = [P \ dE0[i] - dot(dE0[i],t[i])*t[i] for i = 1:length(x)]
         numdE += 1
         # residual, store history
         res = maximum([norm(dE0perp[i],Inf) for i = 1:length(x)])
#         resF = abs(res - log.res_trans[end])
        push!(log, numE, numdE, res, resF)
         if verbose >= 2
            @printf("%4d |   %1.2e\n", nit, res, resF)
         end
        if res <= tol_res # || resF <= tol_res*0
            if verbose >= 1
               println("StringMethod terminates succesfully after $(nit) iterations")
            end
            return x, log
         end
         x -= alpha * dE0perp
         # reparametrise
         ds = [sqrt(dot(x[i+1]-x[i], P, x[i+1]-x[i])) for i=1:length(x)-1]
         s = [0; [sum(ds[1:i]) for i in 1:length(ds)]]
         s /= s[end]; s[end] = 1.
         S = spline(s, x)
         x = [[S[i](s) for i in 1:length(S)] for s in linspace(0., 1.,
                                                                  length(x)) ]
         t = [[derivative(S[i], s) for i in 1:length(S)] for s in
                                                  linspace(0., 1., length(x)) ]
      end
      if verbose >= 1
         println("StringMethod terminated unsuccesfully after $(maxnit) iterations.")
      end
      return x, log
   end

spline_i(x, y, i) =  Spline1D( x, [y[j][i] for j=1:length(y)],
                                    w = ones(length(x)), k = 3, bc = "error" )
spline(x,y) = [spline_i(x,y,i) for i=1:length(y[1])]


spline (generic function with 1 method)

In [57]:
function Base.push!(log::SaddleSearch.IterationLog, numE, numdE, res_trans)
   push!(log.numE, numE)
   push!(log.numdE, numdE)
   push!(log.res_trans, res_trans)
end

In [58]:
V = LJVacancy2D(R = 5.1)
E, dE = objective(V)

X0 = copy(V.Xref); X1 = copy(V.Xref)
X0[:,1] *= 1.; X1[:,1] *= .0
x0 = X0[:, V.Ifree][:]
x1 = X1[:, V.Ifree][:]

P = SaddleSearch.TestSets.precond(V, x0)

# X = SaddleSearch.TestSets.dofs2pos(V, x0)
# plot(X[1,:], X[2,:], "b.")
# plot(X[1,V.Ifree], X[2,V.Ifree], "rx")

180×180 sparse matrix with 2736 Float64 nonzero entries:
	[1  ,   1]  =  367.022
	[31 ,   1]  =  -0.28125
	[33 ,   1]  =  -0.888889
	[35 ,   1]  =  -0.28125
	[47 ,   1]  =  -0.888889
	[49 ,   1]  =  -72.0
	[51 ,   1]  =  -72.0
	[53 ,   1]  =  -0.888889
	[65 ,   1]  =  -0.28125
	[67 ,   1]  =  -72.0
	⋮
	[177, 179]  =  -72.0
	[179, 179]  =  218.623
	[150, 180]  =  -0.28125
	[152, 180]  =  -0.888889
	[154, 180]  =  -0.28125
	[164, 180]  =  -0.888889
	[166, 180]  =  -72.0
	[168, 180]  =  -72.0
	[176, 180]  =  -0.28125
	[178, 180]  =  -72.0
	[180, 180]  =  218.623

In [101]:
# initial string 
N = 25
xstr =  [ (1-s) * x0 + s * x1  for s in linspace(0, 1, N) ]
s = linspace(0, 1, N); 

S = Array{Dierckx.Spline1D,2}(size(X0,2),2)
S[:] = [Spline1D(s*i, [xstr[j][i] for j=1:length(xstr)], w = ones(length(xstr)), k = 3, bc = "error") for i=1:length(xstr[1])]
# S = transpose(S)

Tstr = [[derivative(S[i,j], s) for j in 1:2, i in 1:size(X0,2)] for s in linspace(0., 1., N) ]
tstr = [Tstr[i]'[:] for i=1:N]
#[[derivative(S[i], s) for i in 1:length(X0)] for s in linspace(0., 1., N) ]

string = StringMethod2(0.0007, 1e-1, 100, I, (P, x) -> P, 2, false)

StringMethod2(0.0007,0.1,100,UniformScaling{Int64}
1*I,#161,2,false)

In [62]:
STRx, STRlog = run2!(string, E, dE, xstr, tstr)
Tstr[:]=tstr
# Xbb = SaddleSearch.TestSets.dofs2pos(V, xbb)
# plot(Xbb[1,:], Xbb[2,:], "b.")
# plot(Xbb[1,V.Ifree], Xbb[2,V.Ifree], "rx")

 nit |  sup|∇E|_∞   
-----|-----------------
   0 |   4.50e+01
   1 |   1.61e+01
   2 |   8.91e+00
   3 |   5.78e+00
   4 |   4.13e+00
   5 |   3.15e+00
   6 |   2.53e+00
   7 |   2.15e+00
   8 |   1.88e+00
   9 |   1.66e+00
  10 |   1.47e+00
  11 |   1.31e+00
  12 |   1.17e+00
  13 |   1.05e+00
  14 |   9.51e-01
  15 |   8.62e-01
  16 |   8.08e-01
  17 |   7.68e-01
  18 |   7.51e-01
  19 |   7.38e-01
  20 |   7.26e-01
  21 |   7.15e-01
  22 |   7.05e-01
  23 |   6.95e-01
  24 |   6.86e-01
  25 |   6.77e-01
  26 |   6.69e-01
  27 |   6.60e-01
  28 |   6.53e-01
  29 |   6.47e-01
  30 |   6.41e-01
  31 |   6.34e-01
  32 |   6.28e-01
  33 |   6.21e-01
  34 |   6.15e-01
  35 |   6.08e-01
  36 |   6.02e-01
  37 |   5.95e-01
  38 |   5.88e-01
  39 |   5.82e-01
  40 |   5.75e-01
  41 |   5.68e-01
  42 |   5.61e-01
  43 |   5.54e-01
  44 |   5.48e-01
  45 |   5.41e-01
  46 |   5.34e-01
  47 |   5.27e-01
  48 |   5.20e-01
  49 |   5.14e-01
  50 |   5.07e-01
  51 |   5.00e-01
  52 |   4.94e-01
 

LoadError: MethodError: Cannot `convert` an object of type Array{Float64,1} to an object of type Array{Float64,2}
This may have arisen from a call to the constructor Array{Float64,2}(...),
since type constructors fall back to convert methods.

In [100]:
X = copy(V.Xref)
X[:] = STRx[1]
anim = @animate for i =1 :25
    X[:] = STRx[i]
    Plots.plot(X[1,:], X[2,:], marker=(10.0,:c), linestyle=:dot, linewidth=0)
    Plots.plot!(X[1,V.Ifree], X[2,V.Ifree], marker=(15.0,:+), linestyle=:dot, linewidth=0, size=(700, 500))
end

Plots.gif(anim, fps=15)

#     Plots.plot!(X0[1,V.Ifree], X0[2,V.Ifree], marker=(15.0,:x), linestyle=:dot, linewidth=0, size=(700, 500))

ImageMagick can be installed by executing `Pkg.add("ImageMagick")`
Will try ffmpeg, but it's lower quality...)


LoadError: could not spawn `ffmpeg -v 0 -framerate 15 -loop 0 -i /var/folders/pc/486zp2n124qdyf99trf7s2rm0000gn/T/tmpwHGYPQ/%06d.png -y /Users/sMakri/string-method/temp/vacancy.gif`: no such file or directory (ENOENT)

In [ ]:
plot(X[1,:], X[2,:], "b.")
plot(X[1,V.Ifree], X[2,V.Ifree], "rx")